In [1]:
import numpy as np

In [2]:
num_voxels = 20

In [3]:
    # fit_x = temp[0]
    # fit_y = temp[1]
    # fit_sigma = temp[2]
    # fit_n =  temp[3]
    # fit_beta = temp[4]
    # fit_baseline = temp[5]

In [4]:
# Define the bounds for each variable
x_bound = (-12.0, 12.0)
y_bound = (-12.0, 12.0)
s_bound = (1/17.811033313241005, 12.0)
b_bound = (1e-8, None)
n_bound = (1e-1, 4.0)
u_bound = (-12.0,12.0) # Taking Any arbitrary bounds for now

In [5]:
# Generate random voxel values within the specified bounds using NumPy
x_values = np.random.uniform(x_bound[0], x_bound[1], num_voxels)
y_values = np.random.uniform(y_bound[0], y_bound[1], num_voxels)
sigma_values = np.random.uniform(s_bound[0], s_bound[1], num_voxels)
beta_values = np.random.uniform(b_bound[0], b_bound[1] if b_bound[1] is not None else 1.0, num_voxels)
n_values = np.random.uniform(n_bound[0], n_bound[1], num_voxels)
u_values = np.random.uniform(u_bound[0], u_bound[1], num_voxels)

In [6]:
voxels = np.column_stack((x_values, y_values, sigma_values, n_values, beta_values, u_values))

In [7]:
voxels[0][0]

0.8276729979708826

In [8]:
print(voxels)

[[  0.827673     3.79489988   0.62685362   3.11479199   0.43065456
   -2.49695567]
 [  5.61221948  -6.56639211   8.48018973   3.1008803    0.60082478
   10.28012256]
 [ 10.50102309   8.34206017   2.7509294    0.21317989   0.57999188
    7.58452202]
 [ 11.28327302  -0.6958       6.41737001   3.04452174   0.12520241
  -11.5099669 ]
 [  1.51009115  -9.9572608    7.081222     2.25839886   0.83215186
   -2.00190142]
 [ -2.54489429  -3.58706226   6.43339285   2.5100679    0.27578287
   -3.02046113]
 [ -2.90990803   1.3601752    5.00494517   2.23199585   0.73335948
    3.89279954]
 [ 10.47166131  -5.98625562  10.93920851   3.21081524   0.29289214
    1.9141163 ]
 [ -6.93646456   6.8059617    2.31138521   1.44326485   0.97434647
   -8.54725265]
 [ -5.12865393  -7.64675006   2.29260568   1.61725407   0.706781
  -10.14666706]
 [ -8.34900757   3.01617045   4.57371283   0.64865633   0.56340271
   -0.8674396 ]
 [ -0.95727384  10.69883267   1.32994885   3.63035448   0.57533532
   -3.29320804]
 [  0.

In [9]:
import ctypes, multiprocessing
import numpy as np
import sharedmem
from popeye import css
import popeye.utilities as utils
from popeye.visual_stimulus import VisualStimulus, simulate_bar_stimulus
from matplotlib import pyplot

In [10]:
# ### STIMULUS
# ## create sweeping bar stimulus
# thetas = np.array([-1,0,90,180,270,-1]) # in degrees, -1 is blank
# bar = simulate_bar_stimulus(100, 100, 40, 20, thetas, 30, 30, 10)

# ## create an instance of the Stimulus class
# stimulus = VisualStimulus(bar, 50, 25, 0.50, 1.0, ctypes.c_int16)

# ### MODEL
# ## initialize the gaussian model
# model = css.CompressiveSpatialSummationModel(stimulus, utils.double_gamma_hrf)
# model.hrf_delay = 0 #This is additional delay; Note that double_gamma_hrf already has 5 sec delay by default

In [11]:
from scipy.io import loadmat

In [12]:
from popeye.visual_stimulus import VisualStimulus, resample_stimulus

In [13]:
# some params
native_width = 36.3 # assuming this from prisma measurements i made at NYU
viewing_distance = 64.53 # assuming this from prisma measurements i made at NYU
tr_length = 1.3 # 1300 ms per file name
dtype = ctypes.c_int16 # for stimulus
scale_factor = 1 # for speed
resample_factor = 0.85 # for speed

In [14]:
bar = loadmat('Stimuli/bar_stimulus_masks_1300ms_images.mat')['images']
params = loadmat('Stimuli/bar_stimulus_masks_1300ms_params.mat')

In [15]:
bar = resample_stimulus(bar, resample_factor)
stimulus = VisualStimulus(bar, viewing_distance, native_width, scale_factor, tr_length, dtype)

In [16]:
model = css.CompressiveSpatialSummationModel(stimulus, utils.double_gamma_hrf)
model.hrf_delay = 0 #This is additional delay; Note that double_gamma_hrf already has 5 sec delay by default

In [17]:
### FIT
## define search grids
# these define min and max of the edge of the initial brute-force search.
x_grid = (-10, 10)
y_grid = (-10, 10)
s_grid = (0.25, 12)
n_grid = (.1, 4) #exponent

## define search bounds
# these define the boundaries of the final gradient-descent search.
x_bound = (-12.0, 12.0)
y_bound = (-12.0,12.0)
s_bound = (1/stimulus.ppd, 12.0) #smallest sigma is a pixel
b_bound = (1e-8,None)
u_bound = (None,None)
n_bound = (1e-1, 4.0)

In [18]:
grids = (x_grid, y_grid, s_grid, n_grid)
bounds = (x_bound, y_bound, s_bound, n_bound, b_bound, u_bound,)

In [19]:
sigma_values, beta_values, n_values, u_values

(array([ 0.62685362,  8.48018973,  2.7509294 ,  6.41737001,  7.081222  ,
         6.43339285,  5.00494517, 10.93920851,  2.31138521,  2.29260568,
         4.57371283,  1.32994885, 11.95338599,  5.17481369,  1.42591142,
         7.71768147,  3.47302822,  5.35684422,  2.15394716,  3.03687096]),
 array([0.43065456, 0.60082478, 0.57999188, 0.12520241, 0.83215186,
        0.27578287, 0.73335948, 0.29289214, 0.97434647, 0.706781  ,
        0.56340271, 0.57533532, 0.17692439, 0.95670951, 0.30847825,
        0.35554846, 0.53747085, 0.52771756, 0.60977855, 0.354605  ]),
 array([3.11479199, 3.1008803 , 0.21317989, 3.04452174, 2.25839886,
        2.5100679 , 2.23199585, 3.21081524, 1.44326485, 1.61725407,
        0.64865633, 3.63035448, 3.57406958, 1.4855502 , 3.42825278,
        3.66423887, 2.74294209, 0.66237032, 1.32683006, 0.56226421]),
 array([ -2.49695567,  10.28012256,   7.58452202, -11.5099669 ,
         -2.00190142,  -3.02046113,   3.89279954,   1.9141163 ,
         -8.54725265, -10.1466

In [20]:
fitted_data_array = np.empty((num_voxels, 6)) 

In [21]:
for voxel_index, voxel in enumerate(voxels):
    x = voxel[0]
    y = voxel[1]
    sigma = voxel[2]
    n = voxel[3]
    beta = voxel[4]
    baseline = voxel[5]
    data = model.generate_prediction(x, y, sigma, n, beta, baseline)
    data += np.random.uniform(-data.max()/10,data.max()/10,len(data))
    fit = css.CompressiveSpatialSummationFit(model, data, grids, bounds, Ns=3, voxel_index=(1,2,3), auto_fit=True,verbose=2)
    temp = fit.estimate
    fit_x = temp[0]
    fit_y = temp[1]
    fit_sigma = temp[2]
    fit_n =  temp[3]
    fit_beta = temp[4]
    fit_baseline = temp[5]
    print("x: ", x, "Fit x: ", fit_x)
    print("y: ", y, "Fit y: ", fit_y)
    print("sigma: ", x, "Fit sigma: ", fit_sigma)
    print("n: ", n, "Fit n: ", fit_n)
    print("beta: ", beta, "Fit beta: ", fit_beta)
    print("baseline: ", baseline, "Fit baseline: ", fit_baseline)
    print()
    fitted_data_array[voxel_index] = fit.estimate
    

The gridfit solution was [ 0.          0.          6.125       4.          0.42702247 -2.49816502], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=2887   RSQ=1.00  EST=[ 1.3113  4.1101  0.429   4.      0.4308 -2.4982]
x:  0.8276729979708826 Fit x:  1.0545586629863637
y:  3.7948998837094763 Fit y:  3.9725096695340225
sigma:  0.8276729979708826 Fit sigma:  0.8580915674916281
n:  3.114791988129761 Fit n:  4.0
beta:  0.43065455648530526 Fit beta:  0.4308045045808049
baseline:  -2.4969556741239085 Fit baseline:  -2.4981697206856013

The gridfit solution was [ 10.         -10.           6.125        0.1          1.86649281
  10.29675903], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=2459   RSQ=0.62  EST=[ 5.4398  8.3253  3.4756  0.2989  0.5053 10.2968]
x:  5.612219482970595 Fit x:  5.536055545569798
y:  -6.566392113209096 Fit y:  -6.217915665639363
sigma:  5.612219482970595 Fit sigma:  1.900157032101617
n:  3.100880297313382 Fit n:  0.29889466620664845
beta:  0.6008

In [22]:
fitted_data_array

array([[  1.05455866,   3.97250967,   0.85809157,   4.        ,
          0.4308045 ,  -2.49816972],
       [  5.53605555,  -6.21791567,   1.90015703,   0.29889467,
          0.5053039 ,  10.29676218],
       [  9.4143503 ,   7.19806841,   6.51977143,   1.26222685,
          0.55496976,   7.63357883],
       [ 10.5498711 ,  -2.47469019,   0.3255689 ,   4.        ,
          0.0693131 , -11.43745462],
       [  1.47485045,  -9.93993846,   7.1233025 ,   2.27652753,
          0.82701472,  -2.00178665],
       [ -2.24392391,  -3.56065845,   6.80415676,   3.03672964,
          0.27314176,  -3.0182322 ],
       [ -2.76775322,   1.56287731,   5.75355316,   3.26644109,
          0.71140087,   3.88639938],
       [ 10.42765859,  -5.72196878,   9.61987097,   2.57412757,
          0.30082356,   1.90244853],
       [ -7.4883939 ,   5.90958059,   0.30946639,   0.86430211,
          0.82162587,  -8.54132915],
       [ -5.04779774,  -7.65144733,   0.45299587,   0.1       ,
          0.69219857, -10.1

In [23]:
parameters_names = ['x', 'y', 'sigma', 'n', 'beta', 'baseline']

# Calculate Mean Squared Error for each parameter
mse_per_parameter = np.mean((voxels - fitted_data_array) ** 2, axis=0)

# Print the MSE for each parameter
print("Mean Squared Error for Each Parameter:")
for i, mse in enumerate(mse_per_parameter):
    print(f"{parameters_names[i]}: {mse}")

Mean Squared Error for Each Parameter:
x: 0.20727941357692314
y: 0.37603334037293584
sigma: 6.513795223103829
n: 1.1848670657672613
beta: 0.0030028842412495454
baseline: 0.0005875368790586072


In [24]:
def dry_run_uniform(num_voxels):
    print("For ", num_voxels, " voxels with Uniform Noise added to the Data")
    x_bound = (-12.0, 12.0)
    y_bound = (-12.0, 12.0)
    s_bound = (1/17.811033313241005, 12.0)
    b_bound = (1e-8, None)
    n_bound = (1e-1, 4.0)
    u_bound = (-12.0,12.0) # Taking Any arbitrary bounds for now
    # Generate random voxel values within the specified bounds using NumPy
    x_values = np.random.uniform(x_bound[0], x_bound[1], num_voxels)
    y_values = np.random.uniform(y_bound[0], y_bound[1], num_voxels)
    sigma_values = np.random.uniform(s_bound[0], s_bound[1], num_voxels)
    beta_values = np.random.uniform(b_bound[0], b_bound[1] if b_bound[1] is not None else 1.0, num_voxels)
    n_values = np.random.uniform(n_bound[0], n_bound[1], num_voxels)
    u_values = np.random.uniform(u_bound[0], u_bound[1], num_voxels)
    voxels = np.column_stack((x_values, y_values, sigma_values, n_values, beta_values, u_values))
    fitted_data_array = np.empty((num_voxels, 6)) 
    for voxel_index, voxel in enumerate(voxels):
        x = voxel[0]
        y = voxel[1]
        sigma = voxel[2]
        n = voxel[3]
        beta = voxel[4]
        baseline = voxel[5]
        data = model.generate_prediction(x, y, sigma, n, beta, baseline)
        data += np.random.uniform(-data.max()/10,data.max()/10,len(data))
        fit = css.CompressiveSpatialSummationFit(model, data, grids, bounds, Ns=3, voxel_index=(1,2,3), auto_fit=True,verbose=2)
        temp = fit.estimate
        fit_x = temp[0]
        fit_y = temp[1]
        fit_sigma = temp[2]
        fit_n =  temp[3]
        fit_beta = temp[4]
        fit_baseline = temp[5]
        print("x: ", x, "Fit x: ", fit_x)
        print("y: ", y, "Fit y: ", fit_y)
        print("sigma: ", sigma, "Fit sigma: ", fit_sigma)
        print("n: ", n, "Fit n: ", fit_n)
        print("beta: ", beta, "Fit beta: ", fit_beta)
        print("baseline: ", baseline, "Fit baseline: ", fit_baseline)
        print()
        fitted_data_array[voxel_index] = fit.estimate
    parameters_names = ['x', 'y', 'sigma', 'n', 'beta', 'baseline']
    
    # Calculate Mean Squared Error for each parameter
    mse_per_parameter = np.mean((voxels - fitted_data_array) ** 2, axis=0)
    
    # Print the MSE for each parameter
    print("Mean Squared Error for Each Parameter:")
    for i, mse in enumerate(mse_per_parameter):
        print(f"{parameters_names[i]}: {mse}")


In [25]:
dry_run_uniform(10)

For  10  voxels with Uniform Noise added to the Data
The gridfit solution was [-10.           0.          12.           2.05         1.46370397
  -6.87253869], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=2687   RSQ=0.99  EST=[ 3.4551  6.1644  1.8822  1.1649  0.891  -6.8725]
x:  -5.863481007848339 Fit x:  -5.864068254411013
y:  -1.9220900066172568 Fit y:  -1.9008205354636636
sigma:  1.5340315953891293 Fit sigma:  2.0315144698786756
n:  0.7126214142919016 Fit n:  1.1648978679221116
beta:  0.8969663972300826 Fit beta:  0.8910211725978437
baseline:  -6.877518405875478 Fit baseline:  -6.872535921971745

The gridfit solution was [ 0.          0.          6.125       4.          0.873595   -9.91358215], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=1616   RSQ=0.96  EST=[ 1.6102  2.9694  2.0893  1.3018  0.855  -9.9136]
x:  -0.1521584958595561 Fit x:  -0.11703055047719343
y:  2.984213956996612 Fit y:  2.9671226474408314
sigma:  2.55938372535573 Fit sigma:  2.38374154

In [26]:
dry_run_uniform(100)

For  100  voxels with Uniform Noise added to the Data
The gridfit solution was [ 10.         -10.           6.125        0.1          0.55024084
  -6.5073418 ], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=2122   RSQ=0.26  EST=[ 5.1772 13.4241 17.5674  0.4666  0.5235 -6.5073]
x:  5.237051304407764 Fit x:  6.017237103407413
y:  -10.478041720464667 Fit y:  -12.0
sigma:  9.666861161983418 Fit sigma:  12.0
n:  0.39073936152213773 Fit n:  0.4666006736821136
beta:  0.4656466698515647 Fit beta:  0.523493452326663
baseline:  -6.494964502063546 Fit baseline:  -6.507341803163381

The gridfit solution was [ 0.          0.          0.25        2.05       -0.01932136 -6.02404744], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=0707   RSQ=0.02  EST=[ 0.      0.      0.2151  2.05    0.     -6.024 ]
x:  -6.498695015345279 Fit x:  0.0
y:  2.0328401362492485 Fit y:  0.0
sigma:  10.776106736941156 Fit sigma:  0.30802763154596247
n:  0.24267586730716886 Fit n:  2.05
beta:  0.1354

In [27]:
#this is without adding any noise
def dry_run_no_noise(num_voxels):
    print("For ", num_voxels, " voxels with no noise added to the data")
    x_bound = (-12.0, 12.0)
    y_bound = (-12.0, 12.0)
    s_bound = (1/17.811033313241005, 12.0)
    b_bound = (1e-8, None)
    n_bound = (1e-1, 4.0)
    u_bound = (-12.0,12.0) # Taking Any arbitrary bounds for now
    # Generate random voxel values within the specified bounds using NumPy
    x_values = np.random.uniform(x_bound[0], x_bound[1], num_voxels)
    y_values = np.random.uniform(y_bound[0], y_bound[1], num_voxels)
    sigma_values = np.random.uniform(s_bound[0], s_bound[1], num_voxels)
    beta_values = np.random.uniform(b_bound[0], b_bound[1] if b_bound[1] is not None else 1.0, num_voxels)
    n_values = np.random.uniform(n_bound[0], n_bound[1], num_voxels)
    u_values = np.random.uniform(u_bound[0], u_bound[1], num_voxels)
    voxels = np.column_stack((x_values, y_values, sigma_values, n_values, beta_values, u_values))
    fitted_data_array = np.empty((num_voxels, 6)) 
    for voxel_index, voxel in enumerate(voxels):
        x = voxel[0]
        y = voxel[1]
        sigma = voxel[2]
        n = voxel[3]
        beta = voxel[4]
        baseline = voxel[5]
        data = model.generate_prediction(x, y, sigma, n, beta, baseline)
        # data += np.random.uniform(-data.max()/10,data.max()/10,len(data))
        fit = css.CompressiveSpatialSummationFit(model, data, grids, bounds, Ns=3, voxel_index=(1,2,3), auto_fit=True,verbose=2)
        temp = fit.estimate
        fit_x = temp[0]
        fit_y = temp[1]
        fit_sigma = temp[2]
        fit_n =  temp[3]
        fit_beta = temp[4]
        fit_baseline = temp[5]
        print("x: ", x, "Fit x: ", fit_x)
        print("y: ", y, "Fit y: ", fit_y)
        print("sigma: ", sigma, "Fit sigma: ", fit_sigma)
        print("n: ", n, "Fit n: ", fit_n)
        print("beta: ", beta, "Fit beta: ", fit_beta)
        print("baseline: ", baseline, "Fit baseline: ", fit_baseline)
        print()
        fitted_data_array[voxel_index] = fit.estimate
    parameters_names = ['x', 'y', 'sigma', 'n', 'beta', 'baseline']
    
    # Calculate Mean Squared Error for each parameter
    mse_per_parameter = np.mean((voxels - fitted_data_array) ** 2, axis=0)
    
    # Print the MSE for each parameter
    print("Mean Squared Error for Each Parameter:")
    for i, mse in enumerate(mse_per_parameter):
        print(f"{parameters_names[i]}: {mse}")


In [28]:
dry_run_no_noise(10)

For  10  voxels with no noise added to the data
The gridfit solution was [  0.         -10.          12.           2.05         0.33690468
   1.10911389], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=1554   RSQ=1.00  EST=[4.892  9.8945 6.869  1.7871 0.284  1.1091]
x:  1.7680421042771393 Fit x:  1.768031420667798
y:  -9.73513645159515 Fit y:  -9.73529621593878
sigma:  9.18182482246356 Fit sigma:  9.182598360042629
n:  1.7868374301973204 Fit n:  1.7870622968561012
beta:  0.2840223325489228 Fit beta:  0.2840120106716683
baseline:  1.1091138878347113 Fit baseline:  1.1091136842098097

The gridfit solution was [-10.         -10.          12.           4.           0.62913975
   1.35418439], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=1468   RSQ=1.00  EST=[ 3.8649 14.5004  5.7267  3.5258  0.6061  1.3542]
x:  -10.869573454126787 Fit x:  -10.869620248323319
y:  -9.597531724556736 Fit y:  -9.597562281177655
sigma:  10.752941314275962 Fit sigma:  10.75298945635208
n:

In [29]:
dry_run_no_noise(100)

For  100  voxels with no noise added to the data
The gridfit solution was [-10.          10.           6.125        0.1          1.3834334
   9.74685045], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=1634   RSQ=1.00  EST=[2.5176 5.1504 6.6979 0.1743 0.6738 9.7469]
x:  -4.164510902766277 Fit x:  -4.179820221008201
y:  2.9956279299933506 Fit y:  3.009300399307157
sigma:  4.106193487003024 Fit sigma:  2.796150917397301
n:  0.3635895294896999 Fit n:  0.17427838289352982
beta:  0.6754111764451767 Fit beta:  0.6738477533865894
baseline:  9.746850453804278 Fit baseline:  9.746858461371685

The gridfit solution was [ 0.         10.          6.125       4.          0.17542154 -9.47056349], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=1412   RSQ=1.00  EST=[ 1.2726 10.8816  4.179   3.2616  0.22   -9.4706]
x:  3.1969419029964214 Fit x:  3.1969302213381017
y:  10.401530780613232 Fit y:  10.401339280232337
sigma:  7.547249814574936 Fit sigma:  7.547288037112803
n:  3.2616

In [30]:
#this is adding Gaussian Noise any noise
def dry_run_Gaussian(num_voxels):
    print("For ", num_voxels, " voxels with Gaussian Noise added to the data")
    x_bound = (-12.0, 12.0)
    y_bound = (-12.0, 12.0)
    s_bound = (1/17.811033313241005, 12.0)
    b_bound = (1e-8, None)
    n_bound = (1e-1, 4.0)
    u_bound = (-12.0,12.0) # Taking Any arbitrary bounds for now
    # Generate random voxel values within the specified bounds using NumPy
    x_values = np.random.uniform(x_bound[0], x_bound[1], num_voxels)
    y_values = np.random.uniform(y_bound[0], y_bound[1], num_voxels)
    sigma_values = np.random.uniform(s_bound[0], s_bound[1], num_voxels)
    beta_values = np.random.uniform(b_bound[0], b_bound[1] if b_bound[1] is not None else 1.0, num_voxels)
    n_values = np.random.uniform(n_bound[0], n_bound[1], num_voxels)
    u_values = np.random.uniform(u_bound[0], u_bound[1], num_voxels)
    voxels = np.column_stack((x_values, y_values, sigma_values, n_values, beta_values, u_values))
    fitted_data_array = np.empty((num_voxels, 6)) 
    for voxel_index, voxel in enumerate(voxels):
        x = voxel[0]
        y = voxel[1]
        sigma = voxel[2]
        n = voxel[3]
        beta = voxel[4]
        baseline = voxel[5]
        data = model.generate_prediction(x, y, sigma, n, beta, baseline)
        std_dev = 0.1  #trying out with this value
        data += np.random.normal(loc=0, scale=std_dev, size=len(data))
        fit = css.CompressiveSpatialSummationFit(model, data, grids, bounds, Ns=3, voxel_index=(1,2,3), auto_fit=True,verbose=2)
        temp = fit.estimate
        fit_x = temp[0]
        fit_y = temp[1]
        fit_sigma = temp[2]
        fit_n =  temp[3]
        fit_beta = temp[4]
        fit_baseline = temp[5]
        print("x: ", x, "Fit x: ", fit_x)
        print("y: ", y, "Fit y: ", fit_y)
        print("sigma: ", sigma, "Fit sigma: ", fit_sigma)
        print("n: ", n, "Fit n: ", fit_n)
        print("beta: ", beta, "Fit beta: ", fit_beta)
        print("baseline: ", baseline, "Fit baseline: ", fit_baseline)
        print()
        fitted_data_array[voxel_index] = fit.estimate
    parameters_names = ['x', 'y', 'sigma', 'n', 'beta', 'baseline']
    
    # Calculate Mean Squared Error for each parameter
    mse_per_parameter = np.mean((voxels - fitted_data_array) ** 2, axis=0)
    
    # Print the MSE for each parameter
    print("Mean Squared Error for Each Parameter:")
    for i, mse in enumerate(mse_per_parameter):
        print(f"{parameters_names[i]}: {mse}")


In [31]:
dry_run_Gaussian(100)

For  100  voxels with Gaussian Noise added to the data
The gridfit solution was [10.          0.          6.125       4.          0.04371585 -4.4843318 ], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=1771   RSQ=0.57  EST=[ 0.3147 12.2853  1.7003  2.9637  0.0539 -4.4843]
x:  11.900267568123802 Fit x:  11.682109720880783
y:  3.6557917030626434 Fit y:  3.8022360465258283
sigma:  2.872840858053054 Fit sigma:  2.92713459796885
n:  3.686250437992948 Fit n:  2.9637139802338868
beta:  0.050973059860370186 Fit beta:  0.05385405830304204
baseline:  -4.482951553175398 Fit baseline:  -4.484331858422804

The gridfit solution was [-10.          10.          12.           2.05         0.32148058
  -0.71375196], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=2550   RSQ=0.97  EST=[ 2.394   7.4847  1.0474  3.0928  0.2338 -0.7138]
x:  -5.503311094857784 Fit x:  -5.488943024631812
y:  5.105087638147172 Fit y:  5.088402069636312
sigma:  1.6593622022737418 Fit sigma:  1.84207969844

In [32]:
import cProfile

In [33]:
cProfile.run('dry_run_Gaussian(100)')

For  100  voxels with Gaussian Noise added to the data
The gridfit solution was [-10.         -10.           6.125        2.05         0.67373958
  -8.63310657], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=1407   RSQ=0.99  EST=[ 4.0266 13.4284  3.7418  2.8445  0.6537 -8.6331]
x:  -8.38829827596245 Fit x:  -8.504061641293267
y:  -10.397594736486361 Fit y:  -10.392473439087528
sigma:  6.105327237214793 Fit sigma:  6.310756682250604
n:  2.6607398875704367 Fit n:  2.844497795917188
beta:  0.6518248657626587 Fit beta:  0.6536719285062697
baseline:  -8.637344353543002 Fit baseline:  -8.63310875254777

The gridfit solution was [  0.        -10.         12.          2.05        0.0375271  10.1517487], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=2232   RSQ=0.15  EST=[ 5.2093  5.1196  6.5199  3.0003  0.0442 10.1518]
x:  3.6436074098438986 Fit x:  2.4406174769484914
y:  -4.744154704035635 Fit y:  -4.50036837660958
sigma:  9.246612887168775 Fit sigma:  11.293367038695

In [34]:
from line_profiler import LineProfiler

In [35]:
profiler = LineProfiler()
profiler.add_function(dry_run_Gaussian)

# Profile the function with input
profiler_wrapper = profiler(dry_run_Gaussian)
result = profiler_wrapper(100)
profiler.print_stats()

For  100  voxels with Gaussian Noise added to the data
The gridfit solution was [10.          0.         12.          2.05        0.87707048  1.46167595], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=1531   RSQ=0.98  EST=[0.1797 7.4539 4.2803 1.9647 0.5928 1.4617]
x:  7.306415251294961 Fit x:  7.33388194894024
y:  1.3670247390630372 Fit y:  1.3323112748067059
sigma:  6.083409382217668 Fit sigma:  5.9996632784841415
n:  2.037823508027683 Fit n:  1.964709175815453
beta:  0.5903189261778846 Fit beta:  0.5928260029562707
baseline:  1.4601743739486714 Fit baseline:  1.4616711935601514

The gridfit solution was [-10.          10.          12.           4.           0.40436857
 -11.233313  ], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=1537   RSQ=0.98  EST=[  2.5148  10.2962   2.3227   4.       0.3303 -11.2333]
x:  -8.460212039892568 Fit x:  -8.338982681522323
y:  5.938784280087393 Fit y:  6.039318240373364
sigma:  4.662760554996371 Fit sigma:  4.645357020399803
n